In [1]:
# pip install python-dotenv
# pip install psycopg2-binary
# pip install selenium
# pip install selenium-manager

In [2]:
import os
import requests
from dotenv import load_dotenv
import psycopg2
import re
import time
from db_connect import db_connect
from db import save_cafe_db,fetch_all_cafe,cafe_region_db
from congestion import compute_congestion_score



#env 파일 환경변수 로드
# load_dotenv() 

# # KaKao API 설정
# KAKAO_API_KEY = os.getenv("KAKAO_REST_API_KEY")
# headers = { "Authorization": f"KakaoAK {KAKAO_API_KEY}" }
# url = "https://dapi.kakao.com/v2/local/search/keyword.json"

In [ ]:
# 지역 별 카페 DB에 저장
cafe_region_db()

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

cur, conn = db_connect()

# 크롤링
def crawl_place_info(place_url):
    
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    try:
        driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        driver.get(place_url)
        time.sleep(5)
        
        print("브라우저 실행 성공")
        wait = WebDriverWait(driver,12)
    
        blog_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'a.link_blog')))
        text = blog_element.get_attribute("textContent")
        if text:
            print("블로그 텍스트:",text.strip())
            match = re.search(r'블로그\s*(\d+)',text.strip())
            if match:
                print("크롤링 성공",match.group(1))
                return int(match.group(1))
            else:
                print("리뷰 없음")
                return 0
        else:
            print("정규식 불일치 - 리뷰 없음")
            return 0
    except Exception as e:
        print("크롤링 실패",e)
        return None
    
    finally:
        driver.quit()

cur.execute("SELECT kakao_id, place_url FROM cafe")
rows = cur.fetchall()

for cafe_id, place_url in rows:
    review_count = crawl_place_info(place_url)
    if review_count is None:
        reiview_count = 0
    print(f"[{cafe_id}] 블로그 리뷰 수: {review_count}")
    
    # 블로그 리뷰 기준으로 혼잡도 계산
    congestion_score = compute_congestion_score(review_count)
    
    cur.execute("""
        UPDATE cafe
        SET review_count = %s,
            congestion_score = %s
        WHERE kakao_id = %s
        """,(review_count,congestion_score,cafe_id))
    print(f"[{cafe_id}] DB 저장 완료")
    print(f"혼잡도 : {congestion_score}")
    conn.commit()
    
    time.sleep(1)
    
cur.close()
conn.close()

DB connect 성공
브라우저 실행 성공
블로그 텍스트: 블로그398개
크롤링 성공 398
[1297001441] 블로그 리뷰 수: 398
[1297001441] DB 저장 완료
혼잡도 : 7
브라우저 실행 성공
블로그 텍스트: 블로그148개
크롤링 성공 148
[1012984730] 블로그 리뷰 수: 148
[1012984730] DB 저장 완료
혼잡도 : 5
브라우저 실행 성공
블로그 텍스트: 블로그281개
크롤링 성공 281
[280071417] 블로그 리뷰 수: 281
[280071417] DB 저장 완료
혼잡도 : 6
브라우저 실행 성공
블로그 텍스트: 블로그163개
크롤링 성공 163
[298115044] 블로그 리뷰 수: 163
[298115044] DB 저장 완료
혼잡도 : 5
브라우저 실행 성공
블로그 텍스트: 블로그54개
크롤링 성공 54
[1509907113] 블로그 리뷰 수: 54
[1509907113] DB 저장 완료
혼잡도 : 4
브라우저 실행 성공
블로그 텍스트: 블로그20개
크롤링 성공 20
[23634701] 블로그 리뷰 수: 20
[23634701] DB 저장 완료
혼잡도 : 3
브라우저 실행 성공
블로그 텍스트: 블로그21개
크롤링 성공 21
[1907205470] 블로그 리뷰 수: 21
[1907205470] DB 저장 완료
혼잡도 : 3
브라우저 실행 성공
블로그 텍스트: 블로그49개
크롤링 성공 49
[17884744] 블로그 리뷰 수: 49
[17884744] DB 저장 완료
혼잡도 : 3
브라우저 실행 성공
블로그 텍스트: 블로그165개
크롤링 성공 165
[14666479] 블로그 리뷰 수: 165
[14666479] DB 저장 완료
혼잡도 : 5
브라우저 실행 성공
블로그 텍스트: 블로그59개
크롤링 성공 59
[1429324904] 블로그 리뷰 수: 59
[1429324904] DB 저장 완료
혼잡도 : 4
브라우저 실행 성공
블로그 텍스트: 블로그1338개
크롤링 성공 1338
[1116018239] 블로그

TypeError: '>=' not supported between instances of 'NoneType' and 'int'